In [ ]:
# Poymer simulation code
# Nikita Dmitrieff, Parisa Kara, Clara Andiazabal, Samuel Kill and Peter Akinshin

import numpy as np
import matplotlib.pyplot as plt
 
##################################################   INPUTS   #################################################

k = (1.3799999999999998e-23)
aa = 1
totalEnergy = 0
number1 = 25
La1 = 5
Lb1 = 10 - La1 
number2 = 25
La2 = La1 
Lb2 = Lb1 
Lbox = 40
mySystem = { "Type1" : {"number" : number1, "La" : La1, "Lb" : Lb1, "Lbox": Lbox}, "Type2" : {"number" : number2, "La" : La2 , "Lb" : Lb2, "Lbox": Lbox} } 
sigma_A = 1 
sigma_B = sigma_A / aa
sigma_AB = ( sigma_A + sigma_B ) / 2
eps_AA = 5 * k
eps_BB = 5 * k
eps_AB = 1 * k
dMax = 0.2 * sigma_A
dMax2 = 2 * dMax
listKeys = list( mySystem.keys() )
kspring = 1 # No value indicated for this variable
dictCell = {}
totCell = []

# Calculates the largest rcutoff there is to make the cells in the box

if sigma_A >= sigma_B:
    rcutoffMax = 2.5 * sigma_A
else:
    rcutoffMax = 2.5 * sigma_B

side = int( Lbox/rcutoffMax ) + 1

class system():
    """The class system() stores all the information concerning the different polymers in the system"""
    def __init__ ( self, i, mySystem ):        
        self.nPoly = mySystem[ listKeys[ i ] ][ 'number' ]            # Number of polymer of this type
        self.nBeadsA = mySystem[ listKeys[ i ] ][ 'La' ]              # Number of beads A for each polymer of this type
        self.nBeadsB = mySystem[ listKeys[ i ] ][ 'Lb' ]              # Number of beads B for each polymer of this type
        self.nBeads = ( self.nBeadsA + self.nBeadsB )                 # Number of beads per polymer
        self.totBeads = self.nPoly * ( self.nBeadsA + self.nBeadsB )  # Number of total beads 
        self.position = Lbox * np.random.rand( self.totBeads, 2 )     # 2D array with 2 columns of n rows with n number of beads in total       
        self.myType = [] # List of AAAAA then BBBBB
        for i in range( self.nPoly ):
            for j in range( self.nBeads ):
                if j < self.nBeadsA:
                    self.myType.append( 'A' )
                else:
                    self.myType.append( 'B' )                       
        self.bonded = []
        for i in range( self.nPoly ):
            count = 0
            while count < self.nBeads - 1:
                n = i * self.nBeads + count
                self.bonded.append( (n, n+1 ) )
                count += 1                          
        self.Beads = []         
        for i in range( self.totBeads ):
            bonds = [] # This list is reset over and over, its only purpose is to help get another list of all the bonds that are within the type of polymer
            for bb in self.bonded:
                if i in bb:
                    for kk in bb:
                        if kk != i:
                            bonds.append( kk )                    
            myB = bead( self.position[ i ], bonds, self.myType[ i ] )
            self.Beads.append( myB ) # Used to call upon the beads for the 'bead' class     
        return  
    
    def energy( self, typePoly ):
        """ energy() is a function that calculates the energy of the type of polymer (To calculate the total energy we need to sum the energy of all the types of polymers, which is done later)"""
        ELJ = 0
        self.Eharmonic = 0
        self.ELJ = 0        
        for aa in mySystem[ listKeys[ typePoly ] ].bonded:     # This loop calculates the Harmonic energy in the system by looking into the self.bonded list that gives all the bonds there are in the polymer type, such as (0, 1) or (15, 16) but not (10, 11) because these two aren't bonded          
            r = length( aa[ 0 ], typePoly, aa[ 1 ], typePoly ) # The length() function calculates the distance between any 2 given beads 
            self.Eharmonic +=  1/2 * kspring * ( r ** 2 )                          
        for bead in range( mySystem[ listKeys [ typePoly ] ].totBeads ): # This loop takes every single bead in the polymer type (bead 0, then 1, then 2...) and calculates the corresponding ELJ
            for nbCell in range( len( dictCell ) ):                      # This loop identifies in which cell number is the bead considered at the moment so it takes every cell and doesn't stop until it finds the bead in one
                if [ listKeys[ typePoly ], bead ] in dictCell[ nbCell ]: # When the bead is found then it looks if the cell is in a position where the findNighbours function can be called.
                    ELJ = 0                                              # The findNeighbours function is a function that calls the neighbouring cells of a single cell, but if the cell is in the top row or bottom row then it won't be able to call upon them due to the way the function has been coded (see the function for more details)
                    if ( side ) ** 2 - side - 2 >= nbCell >= side + 1:   # So if it's not in the top or bottom row, then the cell List method can be used 
                        ELJ += calcELJ( bead, typePoly, 1 ) / 2          # Here the calcELJ function is called upon, that calculates the ELJ of a bead via two ways. Either the "trivial" way (tries everybead in the system), which is assigned the value 0; or using the cell list method (which is much faster), which is assigned the number 1
                    elif ( nbCell > ( side ) ** 2 - side - 2 ) or ( nbCell < side + 1 ): # There it verifies that the cell is in the top or bottom row
                        ELJ += calcELJ( bead, typePoly, 0 ) / 2          # Calculates ELJ the "long" way (see later to understand how the function works)
                    self.ELJ += ELJ                                      # Dividing by 2 here because each bond is accounted for twice
        self.totEnergy = self.Eharmonic + self.ELJ
        self.avgEnergy = self.totEnergy / mySystem[ listKeys[ typePoly ] ].totBeads
        return self.totEnergy
    
class bead():
    """The class bead() thus allows the identity (position, type, bonds) of the bead to be accessed either by this class or the system class"""
    def __init__ ( self, position, bonds, myType ):
        self.position = position
        self.myType = myType
        self.bonded = bonds  
    
def evolveOneStep():
    """The function evolveOneStep() perform one step of the Monte Carlo simulation """
    randomPolyType = int( np.random.rand() * len( mySystem ) )                              # Selection of a random poly type                           
    randomBead = int( np.random.rand() * mySystem[ listKeys[ randomPolyType ] ].totBeads )  # Selection of a random bead in the poly type
    randomDirection = int( np.random.rand() * 2 )                                           # Selection of a direction (along the x or y axis)
    initialEnergy = calcEnergyBead( randomBead, randomPolyType )                            # Calculation of the energy relative to the bead selected for comparison after the displacement (for DE)
    randomDisplacement = ( np.random.rand() * dMax2 ) - dMax                                # Generation of a random displacement
    if randomDirection == 0 : # Arbitrarily, let's say X
        if ( 0 < mySystem[ listKeys[ randomPolyType ] ].Beads[ randomBead ].position[ 0 ] + randomDisplacement < Lbox ): # Making sure that it stays within the 40 x 40 box
            mySystem[ listKeys[ randomPolyType ] ].Beads[ randomBead ].position[ 0 ] += randomDisplacement               # Modification of the position in X       
    elif ( 0 < mySystem[ listKeys[ randomPolyType ] ].Beads[ randomBead ].position[ 1 ] + randomDisplacement < Lbox ):   # Arbitrarily, let's say Y
        mySystem[ listKeys[ randomPolyType ] ].Beads[ randomBead ].position[ 1 ] += randomDisplacement                   # Modification of the position in Y 
    finalEnergy = calcEnergyBead( randomBead, randomPolyType ) # Calculation of the final energy relative to the bead (after random displacement)
    DE = finalEnergy - initialEnergy                           # Calculation of DE  
    if DE > 0 :                                                # If the displacement results in an increase in energy, some calculations are used to see if the new configuration is acceptable or not
        a = np.exp( - DE / ( 1.3799999999999998e-23 ) ) 
        X = np.random.rand()                                   # Generating a random number X between 0 and 1, if X >= a then the old configuration is used, if not then we stick with the new configuration (the higher the increase in energy, the lower the chance of this happening)
        if X >= a :                                                   
            if randomDirection == 0:             # Change in X
                mySystem[ listKeys[ randomPolyType ] ].Beads[ randomBead ].position[ 0 ] -= randomDisplacement         
            elif randomDirection == 1:           # Change in Y
                mySystem[ listKeys[ randomPolyType ] ].Beads[ randomBead ].position[ 1 ] -= randomDisplacement 
    assignCellBead( randomBead, randomPolyType ) # Updates dictCell so that it continues to find the correct neighbouring beads for all beads in the system
    return 

def calcEnergyBead( bead, typePoly ):
    """calcEnergyBead() is a function used to reduce the number of calculations in the evolveOneStep function.
    Instead of calculating the energy of the whole system before and after displacement, the energy relative to the bead selected randomly is calculated (Eharmonic and ELJ)"""
    Eharmonic = 0
    ELJ = 0
    for aa in mySystem[ listKeys[ typePoly ] ].Beads[ bead ].bonded :
        r = length( bead, typePoly, aa, typePoly)
        Eharmonic += 1/2 * kspring * ( r ** 2 )   
    for nbCell in range( len( dictCell ) ):
        if [ listKeys[ typePoly ], bead ] in dictCell[ nbCell ]:
            if ( side + 0 ) ** 2 - side - 2 >= nbCell >= side + 1: # Verifies the cell isn't in the top or bottom row of the box (in this case the calcELJ() function is coded to treat them differntly)
                ELJ += calcELJ( bead, typePoly, 1 )                # The reason it isn't divided by 2 is because either the whole ELJ bond is created between 2 beads or it is destroyed
            else:
                ELJ += calcELJ( bead, typePoly, 0 )                # Since it is in the top or bottom row, it will need to use a different type of calculation (see the 0 instead of the 1 earlier at the end )
    return ( ELJ + Eharmonic )

def whatCutoff( bead1, typePoly1, bead2, typePoly2 ):    
    """WhatCutoff() calculates the rcutoff between any 2 given beads with respect to their types"""
    if ( mySystem[ listKeys[ typePoly1 ] ].Beads[ bead1 ].myType == 'A' ) and ( mySystem[ listKeys[ typePoly2 ] ].Beads[ bead2 ].myType == 'A' ):                            
            rcutoff = 2.5 * sigma_A
    elif ( ( mySystem[ listKeys[ typePoly1 ] ].Beads[ bead1 ].myType == 'B' ) and ( mySystem[ listKeys[ typePoly2 ] ].Beads[ bead2 ].myType == 'A' ) ) or ( ( mySystem[ listKeys[ typePoly1 ] ].Beads[ bead1 ].myType == 'A' ) and ( mySystem[ listKeys[ typePoly2 ] ].Beads[ bead2 ].myType == 'B' ) ):             
            rcutoff = 2.5 * sigma_AB                      
    elif ( mySystem[ listKeys[ typePoly1 ] ].Beads[ bead1 ].myType == 'B' ) and ( mySystem[ listKeys[ typePoly2 ] ].Beads[ bead2 ].myType == 'B' ):  
            rcutoff = 2.5 * sigma_B
    return rcutoff 

def length( bead1, typePolyBead1, bead2, typePolyBead2 ):
    """length() calculates the distance between any 2 given beads"""
    x_i = mySystem[ listKeys[ typePolyBead1 ] ].Beads[ bead1 ].position
    x_j = mySystem[ listKeys[ typePolyBead2 ] ].Beads[ bead2 ].position
    r_vector = x_i - x_j # Vector r 
    r_module = np.sqrt( r_vector[ 0 ] ** 2 + r_vector[ 1 ] ** 2 )
    return r_module

def calcEnergy():
    """calcEnergy() is a function that calculates the total energy of the system by summing the ones from the poly types"""
    i = 0
    totalEnergy = 0
    for k in mySystem :
        mySystem[ k ].energy( i )
        totalEnergy += mySystem[ k ].energy( i )
        i += 1 
    return totalEnergy    

def calcTotAvEnergy():
    """calcTotAvEnergy() is a function that calculates the average energy in the system (Total energy / number of bead in the system)"""
    totAvEnergy = 0
    totalEnergy = 0
    totalBeads = 0
    i = 0
    for k in mySystem :
        mySystem[ k ].energy( i )
        totalEnergy += mySystem[ k ].energy( i )
        totalBeads += mySystem[ k ].totBeads
        i += 1 
    totAvEnergy = totalEnergy / totalBeads
    print( totalBeads )
    return totAvEnergy 

def assignCell():
    """assignCell() assigns a cell to each bead of the system according to their coordinates"""
    i = 0
    x1 = []
    y1 = []
    x2 = []
    y2 = []
    for aa in range( side ):
        for bb in range( side ):
            #totCell.append( [ aa, bb ] ) 
            dictCell[ i ] = []           
            for cc in listKeys:
                for dd in range( mySystem[ cc ].totBeads ):
                    X = mySystem[ cc ].Beads[ dd ].position[ 0 ]
                    Y = mySystem[ cc ].Beads[ dd ].position[ 1 ]
                    if mySystem[ cc ].Beads[ dd ].myType == 'A':
                        x1.append( X )
                        y1.append( Y )
                    else:
                        x2.append( X )
                        y2.append( Y )
                    if aa * rcutoffMax < X < ( aa + 1 ) * rcutoffMax :
                        if bb * rcutoffMax < Y < ( bb + 1 ) * rcutoffMax :
                            dictCell[ i ].append( [ cc, dd ] )
            if bb != ( side - 1 ):
                i += 1        
        i += 1
    mapBead( x1, y1, x2, y2 )
    return dictCell

def assignCellBead( bead, typePoly ):
    """assignCellBead() is a function that changes the cell that a bead is in if it went through a random displacement
    First, it checks the old cell that it was in
    Secondly, it searches for its new cell 
    Thirdly, it compares the oldCell with the newCell and check whether they are the same (this means that despite its recent
    displacement the bead stayed in the same cell), or whether they are different (the bead went from one cell to another)
    Fourthly, it changes (or doesn't change) the cell of the bead in the "record" of all the beads' cell in the system : dictCell"""
    for aa in range( len( dictCell ) ):                       # Looks for the old Cell configuration
        if [ listKeys[ typePoly ], bead ] in dictCell[ aa ]:  # When it finds it it "saves" it for comparison later on
            oldCell = aa  
    for aa in range( side ):                                                   # Looks for the new cell of the bead
        for bb in range( side ):
            X = mySystem[ listKeys[ typePoly ] ].Beads[ bead ].position[ 0 ]
            Y = mySystem[ listKeys[ typePoly ] ].Beads[ bead ].position[ 1 ]
            if aa * rcutoffMax < X < ( aa + 1 ) * rcutoffMax :
                if bb * rcutoffMax < Y < ( bb + 1 ) * rcutoffMax :
                    newCell = aa * side + bb # Because aa is in the Y axis and bb in the X, kindly refer to the drawing submitted in the pdf document 
    if oldCell != newCell:
        dictCell[ oldCell ].pop( dictCell[ oldCell ].index( [ listKeys[ typePoly ], bead ] ) )
        dictCell[ newCell ].append( [ listKeys[ typePoly ], bead ] )    
        
def calcELJ( bead, typePoly, typeOfCalc ):
    """# calcELJ() is a function used to calculate the ELJ of a bead given the number of the bead and its polymer type
    It can do it 2 ways :
        1. It can calculate the distance r between the bead and every single other bead and see if it's above or below
    the rcutoff distance (which is determined by the rcutoff function) and calculate the appropriate 
    energy for this interaction
        2. It uses the Cell List technique (which is much faster). How it does it is it searches for the cell number containing
    the given bead and then with the aid of the findNeighbours function it looks for all the the bead's neighbours.
    The calculation of  the energy of the interactions between the neighbours and the bead is done via the same way as before
    but the number of distances to calculate etc. is far less than before. """
    ELJ = 0
    if typeOfCalc == 0: # Tries every single bead in the system
        for j in range( len( mySystem ) ):                           # Looks into every polymer type             
            for bb in range ( mySystem[ listKeys [ j ] ].totBeads ): # Selects every single bead in the type of polymer (these 2 loops help by taking every single bead there is in the system)
                rcutoff = whatCutoff( bead, typePoly, bb, j )        # Calculates the rcutoff distance fitting the 2 beads
                r = length( bead, typePoly, bb, j)                   # Calculates the distance between the 2 beads
                if ( r > rcutoff ):                            
                    ELJ += 0
                elif r == 0:                   # Here the correct ELJ is calculated according to the polymers' type
                    ELJ += 0                            
                elif ( mySystem[ listKeys [ j ] ].myType[ bb ] == 'A' ) and ( mySystem[ listKeys [ typePoly ] ].myType[ bead ] == 'A' ):                            
                    ELJ += 4 * eps_AA * ( ( sigma_A / r ) ** 12 - ( sigma_A / r ) ** 6 )           
                elif ( ( mySystem[ listKeys [ j ] ].myType[ bb ] == 'B' ) and ( mySystem[ listKeys [ typePoly ] ].myType[ bead ] == 'A' ) ) or ( ( mySystem[ listKeys [ j ] ].myType[ bb ] == 'A' ) and ( mySystem[ listKeys [ typePoly ] ].myType[ bead ] == 'B' ) ):             
                    ELJ += 4 * eps_AB * ( ( sigma_AB / r ) ** 12 - ( sigma_AB / r ) ** 6 )                        
                elif ( mySystem[ listKeys [ j ] ].myType[ bb ] == 'B' ) and ( mySystem[ listKeys [ typePoly ] ].myType[ bead ] == 'B' ):                            
                    ELJ += 4 * eps_BB * ( ( sigma_B / r ) ** 12 - ( sigma_B / r ) ** 6 )
    if typeOfCalc == 1:        # Tries only neighbours
        neighbours = findNeighbours( bead, typePoly )
        for typePoly2, bead2 in neighbours:                                            # Only has a few beads to try on
            rcutoff = whatCutoff( bead, typePoly, bead2, listKeys.index( typePoly2 ) ) # Also, calculates the rcutoff
            r = length( bead, typePoly, bead2, listKeys.index( typePoly2 ) )           # And the distance between the 2 beads
            if ( r > rcutoff ):
                ELJ += 0
            elif r == 0:                        # Calculates the correct ELJ according to the polmyers' types
                ELJ += 0
            elif ( mySystem[ typePoly2 ].myType[ bead2 ] == 'A' ) and ( mySystem[ listKeys [ typePoly ] ].myType[ bead ] == 'A' ):                             
                ELJ += 4 * eps_AA * ( ( sigma_A / r ) ** 12 - ( sigma_A / r ) ** 6 )           
            elif ( ( mySystem[ typePoly2 ].myType[ bead2 ] == 'B' ) and ( mySystem[ listKeys [ typePoly ] ].myType[ bead ] == 'A' ) ) or ( ( mySystem[ typePoly2 ].myType[ bead2 ] == 'A' ) and ( mySystem[ listKeys [ typePoly ] ].myType[ bead ] == 'B' ) ):             
                ELJ += 4 * eps_AB * ( ( sigma_AB / r ) ** 12 - ( sigma_AB / r ) ** 6 )                        
            elif ( mySystem[ typePoly2 ].myType[ bead2 ] == 'B' ) and ( mySystem[ listKeys [ typePoly ] ].myType[ bead ] == 'B' ):                            
                ELJ += 4 * eps_BB * ( ( sigma_B / r ) ** 12 - ( sigma_B / r ) ** 6 )
    return ELJ

def mapBead( x1, y1, x2, y2 ):
    """mapBead() is used to display the beads in the system
    x1 and y1 are the coordinates of every bead of type A
    x2 and y2 are the coordinates of every bead of type B
    these values are taken when assigning the cells to the beads in the assignCell() function"""
    plt.plot( x1, y1, 'go', label = 'Beads of type A' )
    plt.legend()
    plt.plot( x2, y2, 'ro', label = 'Beads of type B' )
    plt.legend()
    plt.title( 'Map of the beads in the System' )
    plt.grid( True )
    plt.show()
    
def findNeighbours(bead, typePoly):
    """findNeighbours() is a function who's role is to find the neighbours of a bead, which are all the beads
    located in the same cell or a neighbouring cell. To visualise the way it works kindly take a look at 
    the sketch provided in the supporting pdf document  """
    neighbours = []
    for nbCell in range( len( dictCell ) ):
        if [ listKeys[ typePoly ], bead ] in dictCell[ nbCell ]:
            for bb in dictCell[ nbCell - side - 1 ]:
                neighbours.append(bb)
            for bb in dictCell[ nbCell - side ]:
                neighbours.append(bb)
            for bb in dictCell[ nbCell - side + 1 ]:
                neighbours.append(bb)
            for bb in dictCell[ nbCell - 1 ]:
                neighbours.append(bb)
            for bb in dictCell[ nbCell ]:
                if bb != [ listKeys[ typePoly ], bead ]:
                    neighbours.append(bb)
            for bb in dictCell[ nbCell + 1 ]:
                neighbours.append(bb)
            for bb in dictCell[ nbCell + side - 1 ]:
                neighbours.append(bb)
            for bb in dictCell[ nbCell + side ]:
                neighbours.append(bb)
            for bb in dictCell[ nbCell + side + 1 ]:
                neighbours.append(bb)
    return neighbours

i = 0
for k in mySystem  :
    mySystem[ k ] = system( i, mySystem )
    i += 1  
    
#################################################      OUTPUTS      ################################################### 

assignCell()

print ( "In the beginning, the energy is : {0}".format( calcEnergy() ) )
evolveOneStep()
EnergyList = [] 
t1 = []
t2 = []
nbSteps = 500000
nSample = 5000
Max = 10**99 # Kindly refer to the supporting pdf document (simulation chapter)
for a in range ( nbSteps ):
    b = a / nSample
    c = int( a / nSample )
    if b == c: # Notes the value of the energy every nSample step
        if calcEnergy() >= Max:
            EnergyList.append( Max )
            t1.append( Max / ( len( listKeys ) + 1 ) ) # These two lists record the energy of a single type of polymer
            t2.append( Max / ( len( listKeys ) + 1 ) )
        else:
            EnergyList.append( calcEnergy() )
            t1.append( mySystem[ 'Type1' ].totEnergy )
            t2.append( mySystem[ 'Type2' ].totEnergy )
    evolveOneStep()
    if a == nbSteps / 5:
        print ( "Loading, {0}...".format( a ) )
        print ( "Then it becomes : {0}".format( calcEnergy() ) )
    if a == 2 * ( nbSteps / 5 ):
        print ( "Loading, {0}...".format( a ) )
        print ( "Then it becomes : {0}".format( calcEnergy() ) )
    if a == 3 * ( nbSteps / 5 ):
        #assignCell() # Displays the assembly of the beads at this stage
        print ( "Loading, {0}...".format( a ) )
        print ( "Then it becomes : {0}".format( calcEnergy() ) )
    if a == 4 * ( nbSteps / 5 ):
        nb = 4 * ( nbSteps / 5 )
        print ( "Loading, {0}...".format( a ) )
        print ( "Then it becomes : {0}".format( calcEnergy() ) )
    if a == 4.5 * ( nbSteps / 5 ):
        print ( "Loading, {0}...".format( a ) )
        print ( "Then it becomes : {0}".format( calcEnergy() ) )
    if a == 4.9 * ( nbSteps / 5 ):
        print ( "Loading, {0}...".format( a ) )
        print ( "Then it becomes : {0}".format( calcEnergy() ) )
    if a == 4.95 * ( nbSteps / 5 ):
        print ( "Loading, {0}...".format( a ) )
        print ( "Then it becomes : {0}".format( calcEnergy() ) )


print( )
print( "==================================     VISUAL REPRESENTATION OF THE BEADS IN THE SYSTEM     ===================================")
print( )
assignCell()
print ( )
print( "========================================     TOTAL ENERGY OF THE SYSTEM     ========================================")
print( )
print ( "Total energy of the system is : {0}".format( calcEnergy() ) )
print( )
print( "===========================================     TOTAL AVERAGE ENERGY     ===========================================")
print( )
print( "The total average energy of the System with La = {0} and Lb = {1} with aa = {2} is {3}".format( La1, Lb1, aa, calcTotAvEnergy() ) )
print( )
print( "=====================================     ENERGY OF THE SYSTEM PER MC STEP     =====================================")
print( )

x = [ nSample * i for i in range( int( nbSteps / nSample ) ) ]
plt.plot( x, EnergyList, 'r-', label = "Energy per MC step" )
plt.legend()
plt.show()

# The lines of code below are used to generate a graph for when two types of polymers are used

# plt.plot( x, t1, 'bo-', label = "Energy per MC step for type 1 Poly" )
# plt.legend()
# plt.show()

# plt.plot( x, t2, 'go-', label = "Energy per MC step for type 2 Poly" )
# plt.legend()
# plt.show()

################################################# HEATMAP #################################################

import pandas as pd
import seaborn as sns

#### Heatmap 1: heatmap with the constant k considered
data = pd.read_csv("Data for Heatmap with k.csv") # The data in the csv file was gathered via running 110 simualations to gather data of average energy. A few screenshots of the simualtions are provided in the supporting pdf document
AvEnergy = ((np.asarray(data['AvEnergy'])).reshape(10,11))

result = data.pivot(index='aa', columns='La', values='AvEnergy')

sns.heatmap(result)
plt.title("Heatmap 1 of Average Energy as a function of aa and La \n") # We decided not to  graph as a function of bb due to the fact
# that a value of bb doesn't necessarily correlate to a integer value of La ( La must be an integer as it is the number of 
# type A beads). Furthermore, this way the scale of the graph will be uniform. 

plt.show()

#### Heatmap 2: heatmap with the constant k not considered
data1 = pd.read_csv("Data for Heatmap 1.csv") # The data in the csv file was gathered via running 220 simualations to gather data of average energy.
AvEnergy1 = ((np.asarray(data1['AvEnergy'])).reshape(20,11))

result1 = data1.pivot(index='aa', columns='La', values='AvEnergy')

sns.heatmap(result1, vmin=40, vmax=100)
plt.title("Heatmap 2 of Average Energy as a function of aa and La \n") 

plt.show()

<Figure size 640x480 with 1 Axes>

In the beginning, the energy is : 118927.14124486627
Loading, 100000...
Then it becomes : 42485.70430508509
Loading, 200000...
Then it becomes : 10077.532681329882
